In [1]:
import mat73
import numpy as np
from rigid_transform_3D import rigid_transform_3D

In [2]:
data_dict = mat73.loadmat('sample.mat')

In [3]:
data_dict.keys()

dict_keys(['DIC3DPPresults'])

In [4]:
data_dict['DIC3DPPresults'].keys()

dict_keys(['AllPairsResults', 'DIC2Dinfo', 'Deform', 'Deform_ARBM', 'Disp', 'FaceCentroids', 'FaceCentroids_ARBM', 'FaceColors', 'FaceCorrComb', 'FaceIsoInd', 'FacePairInds', 'Faces', 'PointPairInds', 'Points3D', 'Points3D_ARBM', 'RBM', 'calibration', 'corrComb', 'distortion', 'pairIndices'])

In [5]:
data_dict['DIC3DPPresults']['Faces'].shape

(13368, 3)

In [6]:
facesPy = data_dict['DIC3DPPresults']['Faces'][:10]

In [7]:
facesPy.shape

(10, 3)

In [8]:
facesPy[1]
# an array with float numbers, int needed

array([4958., 4957., 4689.])

In [9]:
facesPyInt = facesPy.astype(int)
facesPyInt[1]
# got int array

array([4958, 4957, 4689])

In [10]:
#sync load file
DIC3D = data_dict['DIC3DPPresults']
#[todo] save results

In [11]:
#get number of images
nImages = len(DIC3D['Points3D']) # DIC3D['Points3D'] is a list
nImages

5

In [12]:
#DIC3D['Points3D'] is a list of 5 arrays
#DIC3D['Points3D'][0] is a list of 1 array, stand for a image
#DIC3D['Points3D'][0][0] is a list sized 6978 in this case
len(DIC3D['Points3D'][0][0])

6978

In [13]:
#pre-allocate 3D-DIC result variables
Points3D_ARBM=[]
Disp_DispVec=[]
Disp_DispMgn=[]
Disp_DispVec_ARBM=[]
Disp_DispMgn_ARBM=[]
FaceCentroids=[]
FaceCentroids_ARBM=[]
FaceCorrComb=[]
FaceIsoInd=[]
RBM_otMat=[]
RBM_ransVec=[]

In [14]:
#Calculating displacements and rigid body motion
FaceCorrComb=[] #init, or it will append with no limit
FaceCentroids=[None]*nImages # declare it with size
Disp_DispVec=[]
Disp_DispMgn=[]

for image in range(nImages):
    #Face correlation coefficient (worst)
    FaceCorrComb.append(np.amax(DIC3D['corrComb'][image][0][facesPyInt-1],1))# find max in each row
    #Compute face centroids
    means = []
    for ifaces in range(facesPy.shape[0]):# in shorted version will be 10, full will be 13368  
        meanOfEachLine = data_dict['DIC3DPPresults']['Points3D'][image][0][facesPyInt[ifaces,:]-1].mean(axis = 0)
        means.append(meanOfEachLine)
        FaceCentroids[image] = means
    #Compute displacements between frames (per point)
    DispVec = data_dict['DIC3DPPresults']['Points3D'][image][0] - data_dict['DIC3DPPresults']['Points3D'][0][0]
    Disp_DispVec.append(DispVec)
    Disp_DispMgn.append(np.sqrt(DispVec[:,0]**2 + DispVec[:,1]**2 + DispVec[:,2]**2))
    
    #Compute rigid body transformation between point clouds
    


In [ ]:
#########################################################################
#############################draft#############################

In [15]:
#Compute rigid body transformation between point clouds
#error message: matrix A is not 3xN, it is 6978x3
[RotMat,TransVec] = rigid_transform_3D(
                    data_dict['DIC3DPPresults']['Points3D'][1][0],
                    data_dict['DIC3DPPresults']['Points3D'][0][0])

Exception: matrix A is not 3xN, it is 6978x3

In [27]:
#change row & column
argA = data_dict['DIC3DPPresults']['Points3D'][1][0]
argA = argA.transpose()
argB = data_dict['DIC3DPPresults']['Points3D'][0][0]
argB = argB.transpose()

In [17]:
# try run rigid_t here------------------------
# find mean column wise
centroid_A = np.mean(argA, axis=1)#array([  2.86630865,  30.67735677, 120.55570519])
centroid_B = np.mean(argB, axis=1)#array([  0.64130487,  25.41937206, 119.06233202]) 1*3 shape

In [18]:
# ensure centroids are 3x1 (necessary when A or B are
# numpy arrays instead of numpy matrices)
centroid_A = centroid_A.reshape(-1, 1) # to 3*1 shape
centroid_B = centroid_B.reshape(-1, 1)

In [19]:
num_rows, num_cols = argA.shape;
# subtract mean
Am = argA - np.tile(centroid_A, (1, num_cols))
Bm = argB - np.tile(centroid_B, (1, num_cols))

In [20]:
H = np.dot(Am,np.transpose(Bm))

In [32]:
# find rotation
U, S, Vt = np.linalg.svd(H)
R = Vt.T * U.T

In [34]:
t = -R*centroid_A + centroid_B


In [43]:
R

array([[2.41191850e-03, 5.78633176e-02, 8.29638051e-01],
       [4.01029559e-02, 6.69495681e-01, 6.61047705e-02],
       [8.07951146e-01, 4.67474028e-02, 5.31523376e-05]])

In [40]:
Vt
#rigid_t ends here---------------------------------

array([[-0.04064423, -0.06882704, -0.99680032],
       [-0.54752799, -0.83296977,  0.07984016],
       [-0.8357997 ,  0.54902112, -0.00382932]])

In [29]:
RotMat,TransVec = rigid_transform_3D(argA,argB)

In [30]:
RotMat

array([[2.41191850e-03, 5.78633176e-02, 8.29638051e-01],
       [4.01029559e-02, 6.69495681e-01, 6.61047705e-02],
       [8.07951146e-01, 4.67474028e-02, 5.31523376e-05]])

In [203]:
#Compute displacements between frames (per point)
DispVec = data_dict['DIC3DPPresults']['Points3D'][1][0] - data_dict['DIC3DPPresults']['Points3D'][0][0] 

In [271]:
# this produce same error
# done debugging
def testfunction(A,B):
    #if len(A) == len(B):
        #print("wellllll")
   # else:
       # print("wrong")
    print(A.shape)
    print(len(B))
testfunction(argA,argB)


(3, 6978)
3


In [215]:
np.sqrt(DispVec[:,0]**2 + DispVec[:,1]**2 + DispVec[:,2]**2)[0:10]

array([21.12711363, 21.12998768, 21.18308559, 21.30089595, 21.31974562,
       21.27991817, 21.33715132, 21.30546194, 21.24607238, 21.31043549])

In [135]:
#compute face centroids
facesPyInt.shape
facesPyInt[1,:]

array([4958, 4957, 4689])

In [136]:
data_dict['DIC3DPPresults']['Points3D'][1][0][facesPyInt[1,:]-1]# frist 1: image, second 1: face(to 13368)

array([[  8.87010205,  47.95226179, 163.64820456],
       [  8.87154231,  47.95528282, 164.5318443 ],
       [  9.80990601,  47.78336433, 163.59180251]])

In [147]:
meanOfEachLine = data_dict['DIC3DPPresults']['Points3D'][1][0][facesPyInt[1,:]-1].mean(axis = 0) 
meanOfEachLine

array([  9.18385012,  47.89696964, 163.92395046])

In [99]:
#read coorComb
DIC3D['corrComb'][0][0][facesPyInt-1] #DIC3D.corrComb{ii}(F); in this case we have 10 instances

array([[1.35222278, 1.36593578, 1.29955775],
       [0.16628634, 0.16344256, 0.19751639],
       [0.16086167, 0.13946344, 0.13863298],
       [1.25526403, 1.25750794, 1.28705816],
       [0.15405274, 0.17574434, 0.1730297 ],
       [0.06320487, 1.52480727, 0.0826026 ],
       [0.21619105, 0.21821001, 0.21504012],
       [1.27900074, 1.26905087, 1.28909456],
       [1.49670128, 1.46146731, 1.46961375],
       [1.27959089, 1.23319027, 1.24494115]])

In [71]:
x = np.amax(DIC3D['corrComb'][1][0][facesPyInt],1) # find max in each row
x

array([1.10297569, 0.20416364, 0.16270319, 1.30087252, 0.18329407,
       1.27172527, 0.22074969, 1.27464799, 1.41727779, 1.29572459])

In [74]:
facesPy

array([[ 610.,  338.,  339.],
       [4958., 4957., 4689.],
       [2248., 1977., 1978.],
       [2210., 2211., 2480.],
       [2011., 2282., 2281.],
       [1021., 1022., 1293.],
       [4496., 4228., 4497.],
       [2469., 2468., 2199.],
       [   7.,  279.,  278.],
       [ 323.,   52.,  324.]])